In [39]:
from scipy import stats
import math

class City:
    def __init__(self, name, population,mean_leave):
        self.name = name
        self.population = population
        self.mean_leave = mean_leave

    def onramp(self, time):
        st_dev = 33
        commuters_percentage = 0.10
        #Calculate the z-score for the normal dist. at the current time and the last time
        z_now = (time-self.mean_leave) / st_dev
        z_last = ((time-1)-self.mean_leave) / st_dev
        #Calculate the probability of a car entering the highway in the last minute
        cdf_now = stats.norm.cdf(z_now)
        cdf_last = stats.norm.cdf(z_last)
        probability = cdf_now - cdf_last
        #Calculate the expected number of cars entering in the last minute
        cars_coming_on = math.floor(probability*commuters_percentage*self.population)
        return cars_coming_on
    
class Road:
    global free_flow_speed, jam_density
    free_flow_speed = 110/60 #110 is the free flow speed in km/h, so 110/60 is km/min
    jam_density = 135
    def __init__(self, length, lanes, onramp_city):
        self.length = length
        self.lanes= lanes
        self.city = onramp_city
        self.vehicle_count = 0
        
    def outflow(self):
        density = self.vehicle_count / (self.lanes * self.length)
        outflow = free_flow_speed*density*(1 - (density/jam_density))
        if (density > jam_density):
            print("MORE THAN JAMMED")
        return outflow
    
    def update(self,time,road_inflow):
        onramp_inflow = self.city.onramp(time)
        outflow = self.outflow()
        self.vehicle_count = math.floor(self.vehicle_count + onramp_inflow + road_inflow - outflow)
        return {'Count':self.vehicle_count,'Onramp':onramp_inflow,'Road Inflow':road_inflow,'Outflow':outflow}

In [22]:
total = 0
stdev = 100
time_values = [s/1000 for s in range(-10000,10000)]

percentages = []
for inc in range(len(time_values)-1):
    cdf_1 = stats.norm.cdf(time_values[inc])
    cdf_2 = stats.norm.cdf(time_values[inc+1])
    diff = cdf_2 - cdf_1
    percentages.append(diff)
print(sum(percentages))

1.0


In [40]:
# odeint performed on SIR model
import numpy as np
import scipy.integrate as integrate 
import matplotlib.pyplot as plt
from pylab import * 

#Note: t = 0 is 5:30 am

#Setup Our 4 cities
cities = [
    City(name="Hamilton",population=747545,mean_leave=100),
    City(name="Burlington",population=183314,mean_leave=115),
    City(name="Oakville",population=193832,mean_leave=135),
    City(name="Mississauga",population=721599,mean_leave=155)
]
#Setup Our 4 road segments
roads = [
    Road(length=11.1, lanes=3, onramp_city=cities[0]),
    Road(length=17.0, lanes=4, onramp_city=cities[1]),
    Road(length=13.8, lanes=4, onramp_city=cities[2]),
    Road(length=20.9, lanes=3, onramp_city=cities[3])
]

run_time=90  # run_time*time_step must equal 600, which will go through one rush hour
             # (std dev 33 with peak time for Hamilton route at 100, so 99% of the population will 
             # leave between 5:30 and  )


def initialize():
    global time, population_results
    time = 0
    # list of the initial car populations
    population_results = [[road.vehicle_count] for road in roads]
    
def observe():
    global time, population_results
    
    #using the road objects vehicle_count to record itself
    for ind in range(len(population_results)):
        population_results[ind].append(cities[ind].onramp(time))
    
def updateall():
    global time
    outflows = [road.outflow() for road in roads]
    print('Time:',time)
    print(roads[0].update(time, 0))
    print(roads[1].update(time,outflows[0]))
    print(roads[2].update(time,outflows[1]))
    print(roads[3].update(time,outflows[2]))
    
    time = time + 1

initialize()

for t in range(run_time):
    updateall()
    observe() 
    
plot(population_results[0], color='red') 
plot(population_results[1], color='blue') 
plot(population_results[2], color='green')
plot(population_results[3], color='magenta')

show()

Time: 0
{'Count': 8, 'Onramp': 8, 'Road Inflow': 0, 'Outflow': 0.0}
{'Count': 0, 'Onramp': 0, 'Road Inflow': 0.0, 'Outflow': 0.0}
{'Count': 0, 'Onramp': 0, 'Road Inflow': 0.0, 'Outflow': 0.0}
{'Count': 0, 'Onramp': 0, 'Road Inflow': 0.0, 'Outflow': 0.0}
Time: 1
{'Count': 16, 'Onramp': 9, 'Road Inflow': 0, 'Outflow': 0.4396566514239743}
{'Count': 0, 'Onramp': 0, 'Road Inflow': 0.4396566514239743, 'Outflow': 0.0}
{'Count': 0, 'Onramp': 0, 'Road Inflow': 0.0, 'Outflow': 0.0}
{'Count': 0, 'Onramp': 0, 'Road Inflow': 0.0, 'Outflow': 0.0}
Time: 2
{'Count': 25, 'Onramp': 10, 'Road Inflow': 0, 'Outflow': 0.8777457248150163}
{'Count': 0, 'Onramp': 0, 'Road Inflow': 0.8777457248150163, 'Outflow': 0.0}
{'Count': 0, 'Onramp': 0, 'Road Inflow': 0.0, 'Outflow': 0.0}
{'Count': 0, 'Onramp': 0, 'Road Inflow': 0.0, 'Outflow': 0.0}
Time: 3
{'Count': 34, 'Onramp': 11, 'Road Inflow': 0, 'Outflow': 1.3687221867624493}
{'Count': 1, 'Onramp': 0, 'Road Inflow': 1.3687221867624493, 'Outflow': 0.0}
{'Count': 0, 

Time: 49
{'Count': 3060, 'Onramp': 267, 'Road Inflow': 0, 'Outflow': 57.41829528338259}
{'Count': 1308, 'Onramp': 29, 'Road Inflow': 57.41829528338259, 'Outflow': 29.131682525951557}
{'Count': 361, 'Onramp': 7, 'Road Inflow': 29.131682525951557, 'Outflow': 10.65625802235758}
{'Count': 109, 'Onramp': 4, 'Road Inflow': 10.65625802235758, 'Outflow': 2.8323210215074375}
Time: 50
{'Count': 3286, 'Onramp': 280, 'Road Inflow': 0, 'Outflow': 53.795236678119544}
{'Count': 1361, 'Onramp': 30, 'Road Inflow': 53.795236678119544, 'Outflow': 30.240061514801997}
{'Count': 387, 'Onramp': 8, 'Road Inflow': 30.240061514801997, 'Outflow': 11.408911153767363}
{'Count': 122, 'Onramp': 5, 'Road Inflow': 11.408911153767363, 'Outflow': 3.1460927803392154}
Time: 51
{'Count': 3530, 'Onramp': 293, 'Road Inflow': 0, 'Outflow': 48.673506116504655}
{'Count': 1410, 'Onramp': 32, 'Road Inflow': 48.673506116504655, 'Outflow': 31.253537368106283}
{'Count': 414, 'Onramp': 8, 'Road Inflow': 31.253537368106283, 'Outflow':

OverflowError: cannot convert float infinity to integer